In [17]:
# draw isosurface

import webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

cf = x**2 + y**2 + z**2 - 0.2**2

def render(data):
    import ngsolve_webgpu.isosurface
    ngsolve_webgpu.isosurface.render_isosurface(data["cf"], data["mesh"])

def draw_geo(geo):
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    webgpu.jupyter.DrawCustom({"cf" : cf, "mesh" : mesh}, render, ["ngsolve_webgpu"])

draw_geo(box)

<IPython.core.display.Javascript object>

In [ ]:
# draw geometry

import webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1)) + Sphere((1,1,1), 0.4)

def render(data):
    import ngsolve_webgpu.geometry
    ngsolve_webgpu.geometry.render_geometry(data["geo"])

def draw_geo(geo):
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    webgpu.jupyter.DrawCustom({"geo" : geo}, render, ["ngsolve_webgpu"])

draw_geo(box)